In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, ReLU, LeakyReLU
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import to_categorical


In [3]:
# Load ECG5000 dataset
train_data = np.loadtxt("ECG5000_TRAIN.txt")
test_data = np.loadtxt("ECG5000_TEST.txt")

X_train = train_data[:, 1:]
y_train = train_data[:, 0].astype(int) - 1

X_val = test_data[:, 1:]
y_val = test_data[:, 0].astype(int) - 1

num_classes = len(np.unique(y_train))

# One-hot encoding
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)


In [ ]:
X_train = (X_train - np.mean(X_train)) / np.std(X_train)
X_val = (X_val - np.mean(X_val)) / np.std(X_val)


In [6]:

  from tensorflow.keras.layers import Input

def build_model(hidden_layers=2, activation="relu", optimizer="adam", lr=0.001):
    model = Sequential()
    model.add(Input(shape=X_train.shape[1:]))
    model.add(Flatten())

    for _ in range(hidden_layers):
        model.add(Dense(128))
        if activation == "relu":
            model.add(ReLU())
        else:
            model.add(LeakyReLU(alpha=0.1))

    model.add(Dense(num_classes, activation="softmax"))

    if optimizer == "adam":
        opt = Adam(learning_rate=lr)
    else:
        opt = SGD(learning_rate=lr)

    model.compile(
        optimizer=opt,
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model



In [7]:
experiments = [
    {"layers": 2, "activation": "relu", "optimizer": "adam"},
    {"layers": 3, "activation": "relu", "optimizer": "adam"},
    {"layers": 2, "activation": "leakyrelu", "optimizer": "adam"},
    {"layers": 2, "activation": "relu", "optimizer": "sgd"},
]

results = []

for exp in experiments:
    model = build_model(
        hidden_layers=exp["layers"],
        activation=exp["activation"],
        optimizer=exp["optimizer"]
    )

    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        epochs=15,
        batch_size=32,
        verbose=0
    )

    results.append({
        "Hidden layers": exp["layers"],
        "Activation": exp["activation"],
        "Optimizer": exp["optimizer"],
        "Validation accuracy": history.history["val_accuracy"][-1]
    })


In [8]:
results_df = pd.DataFrame(results)
results_df


,Hidden layers,Activation,Optimizer,Validation accuracy
0,2,relu,adam,0.934000
1,3,relu,adam,0.941111
2,2,leakyrelu,adam,0.941556
3,2,relu,sgd,0.901333
